## **Image classification** ##

In [2]:

import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import keras.layers as layers
import numpy as np
from keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import joblib
import math

In [3]:
x = []
y = []

path = './content/data'

#For each class
for c in [c for c in os.listdir(path)if c != ".ipynb_checkpoints"]:

    classPath = os.path.join(path,c)

    for image_name in [i for i in os.listdir(classPath) if i !=  ".ipynb_checkpoints"]:

        image = cv2.imread(os.path.join(classPath,image_name))
        image = cv2.resize(image, (256, 256))

        x.append(image)
        y.append(int(c))

y = np.array(y)
x = np.array(x)


In [4]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.15,random_state=1)

In [ ]:
augm = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

X_train = X_train.reshape((len(X_train), 256, 256, 3))
augm.fit(X_train)
augmented_data = augm.flow(X_train, Y_train, batch_size=15)

In [5]:
Y_train = np.asarray(Y_train).astype('float32').reshape((-1,1))
Y_test = np.asarray(Y_test).astype('float32').reshape((-1,1))

In [6]:
#Build neural network

# Net 
model = Sequential()
model.add(Flatten(input_shape=(3024, 4032, 3)))
model.add(Dense(600, activation='relu'))
model.add(Dense(600, activation='relu'))
model.add(Dense(1, activation='sigmoid')) #Output layer should be sigmoid for binary problems (0,1)

# Optimizer setup
model.compile(loss='binary_crossentropy',#Cost function
              optimizer= SGD(learning_rate=0.01),
              metrics=['accuracy'])



# Net training
model_history = model.fit(X_train, Y_train,
                          epochs=130, verbose=1,
                          validation_data=(X_test, Y_test),
                          callbacks=EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True))

: 

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Flatten())

model.add(Dense(64,activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.fit(augmented_data,epochs=100,verbose=1,batch_size=20)

In [ ]:

def getResults(threshold):
  y_pred = (model.predict(X_test,verbose=0) > threshold).astype(int)
  print('\nThreshold: ', threshold)

  print('Accuracy:',accuracy_score(Y_test, y_pred))
  print('Precision:',precision_score(Y_test, y_pred))
  print('Recall:',recall_score(Y_test, y_pred))
  print('Confusion matrix',confusion_matrix(Y_test, y_pred))

In [ ]:
for threshold in [.5,.6,.68]:
  getResults(threshold)

In [ ]:

def isMyDogInTheVideo(video_path,model,threshold):

  #Reading video
  video = cv2.VideoCapture(video_path)

  #Getting video info
  fps = video.get(cv2.CAP_PROP_FPS)
  total_fps = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  video_duration = total_fps / fps


  #Getting the array of each frame and adding it to frames
  frames = []
  frame_num = 1
  secs = []

  while True:
    ret, frame = video.read()
    if not ret:
        break

    frame = cv2.resize(frame, (256, 256))
    frames.append(frame)
    aug_frame = np.expand_dims(frame, axis=0)

    #Predicting for each frame and showing the image
    if model.predict(aug_frame,verbose=0)[0][0] > threshold:

      sec = math.ceil(frame_num / fps)
      secs.append(sec)

      print(f'Lucas was seen in frame number {frame_num}, in second {sec} with a probability of {model.predict(aug_frame,verbose=0)[0][0]}')
      plt.imshow(frame)
      plt.show()



    frame_num += 1

  #Getting secs where Lucas was seen
  secs = list(set(secs))
  print(f'Lucas appeared in seconds: {secs}')



In [ ]:
isMyDogInTheVideo('.path/test_video2.mp4',model,.70)